# Put Price Evaluation for BTCC.B

## Get Prices

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

# URL of the page
url = 'https://www.m-x.ca/en/trading/data/quotes?symbol=BTCC*'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table
table = soup.find('tbody', {'class': 'text-right nowrap'})

# Initialize an empty list to store the data
options_data = []

# Check if the table is found
if table:
    # Iterate over each row in the table
    for row in table.find_all('tr'):
        try:
            # Extract data for each attribute, with error handling
            strike_price = row.find('td', class_='strike_price').text.strip()
            call_bid_price = row.find('td', class_='put bid_price').text.strip()
            call_ask_price = row.find('td', class_='put ask_price').text.strip()

            # Extract expiry date from the data-row attribute
            data_row = json.loads(row['data-row'].replace('&quot;', '"'))
            expiry_date = data_row['call']['expiry_date'] if 'call' in data_row else 'N/A'

            # Append the data to the list
            options_data.append({
                'Date': expiry_date,
                'Strike': strike_price,
                'Bid': call_bid_price,
                'Ask': call_ask_price
            })
        except (AttributeError, KeyError, json.JSONDecodeError) as e:
            print(f"Error extracting data from row: {e}")
else:
    print("Table not found.")

# Convert the list to a DataFrame
df = pd.DataFrame(options_data)

# Convert 'Expiry Date' to datetime and sort by it
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by=['Date', 'Strike'], inplace=True)

# Convert 'Strike' and 'Ask' columns to numeric (float)
df['Strike'] = pd.to_numeric(df['Strike'], errors='coerce')
df['Ask'] = pd.to_numeric(df['Ask'], errors='coerce')
df

,Date,Strike,Bid,Ask
68,2024-03-15,10.0,0,0.0
69,2024-03-15,10.5,0,0.0
70,2024-03-15,11.0,0,0.8
71,2024-03-15,11.5,0,0.0
72,2024-03-15,12.0,0,0.0
...,...,...,...,...
309,2026-01-16,5.0,0,0.0
310,2026-01-16,6.0,0,0.0
311,2026-01-16,7.0,0,0.0
312,2026-01-16,8.0,0,0.0


## Date of Interest

## Data & Params

In [2]:
# All options expire on January 16th 2026

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz

# Retrieve historical data
ticker = "BTCC-B.TO"
start_time = (datetime.now(pytz.timezone('US/Pacific')) - timedelta(days=365*4)).strftime('%Y-%m-%d')
end_time = (datetime.now(pytz.timezone('US/Pacific'))).strftime('%Y-%m-%d')

data = yf.download(ticker, start=start_time, end=end_time, interval="1d")[['Close']]
data['Daily_Return'] = data['Close'].pct_change()

daily_volatility = data['Daily_Return'].std()
annualized_volatility = daily_volatility * np.sqrt(252)

risk_free_rate = 0.045

current_stock_price = data['Close'].iloc[-1]  
print(current_stock_price)

[*********************100%%**********************]  1 of 1 completed

13.869999885559082



C:\Users\falty\AppData\Local\Temp\ipykernel_16420\1888896037.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Daily_Return'] = data['Close'].pct_change()


## Binomial Tree

In [3]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.stats import norm

# Assuming data['Daily_Return'] contains the daily returns
historical_volatility = np.std(np.log(1 + data['Daily_Return'].dropna())) * np.sqrt(252)  # Annualized using trading days in a year
mean_return = np.mean(data['Daily_Return'].dropna()) * 252  # Annualized

historical_skewness = skew(data['Daily_Return'].dropna())
historical_kurtosis = kurtosis(data['Daily_Return'].dropna(), fisher=False)

def binomial_tree_put_option_price(S, K, T_days, r, sigma, N=20_000, dividends=0, skew=0, kurtosis=0, drift=0):
    T = T_days / 252  # Convert days to years
    dt = T / N
    u = np.exp((drift - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt))  # Incorporating drift
    d = 1 / u
    p = (np.exp((r - dividends) * dt) - d) / (u - d)

    p += skew * 0.001 * (1 - 2 * p) + kurtosis * 0.0005 * (1 - 2 * p)
    p = min(max(p, 0), 1)

    prices = S * d**np.arange(N, -1, -1) * u**np.arange(0, N+1, 1)
    option_values = np.maximum(K - prices, 0)  # Changed for put option

    discount_factor = np.exp(-r * dt)
    for i in range(N - 1, -1, -1):
        option_values[:i+1] = (p * option_values[1:i+2] + (1 - p) * option_values[:i+1]) * discount_factor
        # For American options, compare with early exercise
        option_values[:i+1] = np.maximum(option_values[:i+1], K - prices[:i+1])  # Changed for put option

    return option_values[0]

### Look for Undervalued Puts

In [4]:
%%time

N = 5_000  # Number of steps in the binomial tree

for date in df['Date'].unique():
    T_days = (date - pd.Timestamp.now()).days  # Calculate days to expiration
    options_on_date = df[df['Date'] == date]
    options = list(options_on_date[['Strike', 'Ask']].dropna().apply(tuple, axis=1))

    for strike, ask_price in options:
        option_price = binomial_tree_put_option_price(
            current_stock_price, strike, T_days, risk_free_rate, historical_volatility, N,
            dividends=0, skew=historical_skewness, kurtosis=historical_kurtosis, drift=mean_return
        )

        if ask_price < option_price:
            print(f"Underpriced Put Option for expiration {date.strftime('%Y-%m-%d')}, strike {strike}: Calculated Price: ${option_price:.2f}, Ask Price: ${ask_price}")


Underpriced Put Option for expiration 2024-03-15, strike 10.0: Calculated Price: $9.68, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 10.5: Calculated Price: $10.18, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 11.0: Calculated Price: $10.68, Ask Price: $0.8
Underpriced Put Option for expiration 2024-03-15, strike 11.5: Calculated Price: $11.18, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 12.0: Calculated Price: $11.68, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 12.5: Calculated Price: $12.18, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 13.0: Calculated Price: $12.68, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 13.5: Calculated Price: $13.18, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 14.0: Calculated Price: $13.68, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike

KeyboardInterrupt: 

## Monte-Carlo Simulation 

In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pytz
from numba import jit, prange

# Calculate log returns
log_returns = np.log(data['Close'] / data['Close'].shift(1)).dropna()

# Identify jumps
jump_threshold = 3 * np.std(log_returns)  # Example threshold
jumps = log_returns[abs(log_returns) > jump_threshold]

# Estimate lambda (jump intensity)
lambda_ = len(jumps) / (len(log_returns) / 365)  # Number of jumps per year

# Estimate mu_J and sigma_J (jump size parameters)
# We use the jumps themselves, not the ratio to their lagged values
mu_J = np.mean(jumps)
sigma_J = np.std(jumps)

@jit(nopython=True, parallel=True)  # Enable JIT compilation with parallel execution
def monte_carlo_put_option_price(S, K, T_months, r, sigma, lambda_, mu_J, sigma_J, num_paths=10000, num_steps=100, skew=0, kurtosis=0):
    """
    Monte Carlo simulation for European put option pricing with jump diffusion.

    Parameters:
    S (float): Current stock price.
    K (float): Strike price of the option.
    T_months (int): Time to maturity in months.
    r (float): Risk-free interest rate.
    sigma (float): Volatility of the underlying asset.
    lambda_ (float): Jump intensity, representing the frequency of jumps.
    mu_J (float): Mean of the logarithm of the jump size.
    sigma_J (float): Standard deviation of the logarithm of the jump size.
    num_paths (int): Number of simulated paths in the Monte Carlo simulation.
    num_steps (int): Number of time steps in each simulated path.
    skew (float): Skewness of the asset returns. Defaults to 0 for no skew.
    kurtosis (float): Kurtosis of the asset returns. Defaults to 0 for normal kurtosis.

    Returns:
    float: Estimated price of the European put option.
    """
    T = T_months / 365  # Convert maturity from months to years
    dt = T / num_steps  # Time step for simulation
    discount_factor = np.exp(-r * T)  # Discount factor for present value

    payoffs = np.zeros(num_paths)  # Initialize array for option payoffs
    for i in prange(num_paths):  # Parallel loop for each path
        S_t = S  # Initial stock price for this path
        for j in range(num_steps):  # Time step loop
            z = np.random.normal()  # Standard normal random variable
            shock = z + skew * (z ** 2 - 1)  # Adjust random variable for skew and kurtosis
            num_jumps = np.random.poisson(lambda_ * dt)  # Poisson process for number of jumps
            jump_sum = np.sum(np.random.lognormal(mu_J, sigma_J, num_jumps) - 1)  # Sum of jump sizes
            S_t *= np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * shock + jump_sum)  # Stock price update
        payoffs[i] = max(K - S_t, 0)  # Put option payoff for this path

    return discount_factor * np.mean(payoffs)  # Average discounted payoff

### Look for Undervalued Puts

In [ ]:
%%time

for date in df['Date'].unique():
    T_days = (date - pd.Timestamp.now()).days  # Calculate days to expiration
    options_on_date = df[df['Date'] == date]
    options = list(options_on_date[['Strike', 'Ask']].dropna().apply(tuple, axis=1))

    for strike, ask_price in options:
        option_price = monte_carlo_put_option_price(
            current_stock_price, strike, T_days, risk_free_rate, historical_volatility,
            lambda_, mu_J, sigma_J,
            num_paths=100_000, num_steps=T_days * 24,  # Assuming 24 hours in a day for finer granularity
            skew=historical_skewness, kurtosis=historical_kurtosis
        )

        if ask_price < option_price:
            print(f"Underpriced Put Option for expiration {date.strftime('%Y-%m-%d')}, strike {strike}: Calculated Price: ${option_price:.2f}, Ask Price: ${ask_price}")


Underpriced Put Option for expiration 2024-03-15, strike 10.0: Calculated Price: $0.00, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 10.5: Calculated Price: $0.00, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 11.5: Calculated Price: $0.00, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 12.0: Calculated Price: $0.00, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 12.5: Calculated Price: $0.01, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 13.0: Calculated Price: $0.03, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 13.5: Calculated Price: $0.12, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 14.0: Calculated Price: $0.34, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 8.5: Calculated Price: $0.00, Ask Price: $0.0
Underpriced Put Option for expiration 2024-03-15, strike 8.75: Ca